# Second-Order Warming and Beam
## Derivation and Analysis

In [1]:
import numpy as np
from numpy import exp
import sympy as sy
from sympy.matrices import Matrix, MatrixSymbol
from fractions import Fraction as Fr
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
import numpy as np
# Convension is that symbols start with capital letters

In [2]:
# Derivation of WB2 using spatial gradients from a quadratic
# ax^2 + bx + c  so that ddx = 2ax + b, d2/dx2 = 2a
[C, Chi2] = sy.symbols("C, Chi2", real=True)   # The Courant number and the HO limiter
Psi = sy.Matrix(sy.symarray("Psi", (3,)))               # Grid point values at i-2,i-1 and i

def findDdxD2dx2():
    PolyCoeffs = sy.Matrix(sy.symarray("PolyCoeffs", (3,))) # Coefficients a,b,c of the polynomial
    polyM = Matrix([[4,-2,1], [1,-1,1], [0,0,1]])
    PolyCoeffs = polyM.solve(Psi)
    Ddx = PolyCoeffs[1]
    D2dx2 = 2*PolyCoeffs[0]
    print('d/dx =', Ddx, '\nd2/dx2 =', D2dx2)
    return Ddx, D2dx2

WB_Ddx, WB_D2dx2 = findDdxD2dx2()

# From this we can work out the explicit WB3e  increment
WB2e = - C*WB_Ddx + Chi2*C**2/2*WB_D2dx2
print('WB2e increment is', WB2e)
print('WB2e increment as coefficients of grid points is\n', sy.collect(sy.expand(WB2e), Psi))
# The implicit WB2i scheme
WB2i = - C*WB_Ddx - Chi2*C**2/2*WB_D2dx2
print('WB2i increment is ', WB2i)
print('WB2i increment as coefficients of grid points is\n', sy.collect(sy.expand(WB2i), Psi))

# The amplification factor for WB2_AdImEx
[Kdx, Alp] = sy.symbols("Kdx, Alp", real=True)
A_WB2ImEx0 = (1 + (1-Alp)*WB2e)/(1 - Alp*WB2i)
A_WB2ImEx = A_WB2ImEx0.subs({Psi[0] : sy.E**(-2*sy.I*Kdx), Psi[1] : sy.E**(-sy.I*Kdx), Psi[2] : 1})
print('Amplification factor for WB2 ImEx is \n', A_WB2ImEx)

d/dx = Psi_0/2 - 2*Psi_1 + 3*Psi_2/2 
d2/dx2 = Psi_0 - 2*Psi_1 + Psi_2
WB2e increment is C**2*Chi2*(Psi_0 - 2*Psi_1 + Psi_2)/2 - C*(Psi_0/2 - 2*Psi_1 + 3*Psi_2/2)
WB2e increment as coefficients of grid points is
 Psi_0*(C**2*Chi2/2 - C/2) + Psi_1*(-C**2*Chi2 + 2*C) + Psi_2*(C**2*Chi2/2 - 3*C/2)
WB2i increment is  -C**2*Chi2*(Psi_0 - 2*Psi_1 + Psi_2)/2 - C*(Psi_0/2 - 2*Psi_1 + 3*Psi_2/2)
WB2i increment as coefficients of grid points is
 Psi_0*(-C**2*Chi2/2 - C/2) + Psi_1*(C**2*Chi2 + 2*C) + Psi_2*(-C**2*Chi2/2 - 3*C/2)
Amplification factor for WB2 ImEx is 
 ((1 - Alp)*(C**2*Chi2*(1 - 2*exp(-I*Kdx) + exp(-2*I*Kdx))/2 - C*(3/2 - 2*exp(-I*Kdx) + exp(-2*I*Kdx)/2)) + 1)/(-Alp*(-C**2*Chi2*(1 - 2*exp(-I*Kdx) + exp(-2*I*Kdx))/2 - C*(3/2 - 2*exp(-I*Kdx) + exp(-2*I*Kdx)/2)) + 1)
